# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv
import joblib, pickle
import json

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.data.dataset_factory import TabularDatasetFactory

#from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.33.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'capstone-project'

experiment=Experiment(ws, experiment_name)

In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

compute_cluster = 'automl-cluster'

try:
    comp_trget = ComputeTarget(workspace=ws, name=compute_cluster)
    print('Cluster already exists')
except ComputeTargetException:
    compute_confg = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2",max_nodes=4) ## VER
    comp_trget = ComputeTarget.create(ws,compute_cluster,compute_confg)

In [13]:
data_link = 'https://raw.githubusercontent.com/fcgomezr/Udacity-Project-3/main/data%20sets/WA_Fn-UseC_-Telco-Customer-Churn.csv'
input_data = TabularDatasetFactory.from_delimited_files(path=data_link, separator = ",")

In [15]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(input_data)
predict_col = pd.DataFrame(y)
predict_col.head()
complete_data = pd.concat([x,predict_col],axis=1)
complete_data = complete_data.drop("customerID", axis=1)
complete_data.head()

,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,...,gender_Male,InternetService_Fiber optic,InternetService_No,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Contract_One year,Contract_Two year,Churn
0,0,1,0,1,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0,0,0,34,1,0,1,0,1,0,...,1,0,0,0,0,0,1,1,0,0
2,0,0,0,2,1,0,1,1,0,0,...,1,0,0,0,0,0,1,0,0,1
3,0,0,0,45,0,0,1,0,1,1,...,1,0,0,1,0,0,0,1,0,0
4,0,0,0,2,1,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,1


In [16]:
# split data into training and test data
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(complete_data,test_size=0.25)
train_data.head()

,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,...,gender_Male,InternetService_Fiber optic,InternetService_No,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Contract_One year,Contract_Two year,Churn
4955,0,0,0,1,1,0,0,0,0,0,...,1,1,0,0,0,0,1,0,0,1
1898,1,0,0,5,1,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,1
1541,0,0,0,72,1,0,0,0,0,0,...,1,0,1,1,0,0,0,0,1,0
6676,0,1,1,41,1,1,1,1,0,0,...,0,1,0,1,0,0,0,0,0,0
6293,0,0,1,21,1,0,0,0,0,0,...,1,0,1,0,0,0,1,0,1,0


In [17]:
train_data.to_csv("traindata.csv")

In [18]:
data_store = ws.get_default_datastore()
data_store.upload(src_dir='./',target_path='./')
ftrain_data = Dataset.Tabular.from_delimited_files(path = [(data_store, './traindata.csv')])

Uploading an estimated of 12 files
Uploading ./.amlignore
Uploaded ./.amlignore, 1 files out of an estimated total of 12
Uploading ./.amlignore.amltmp
Uploaded ./.amlignore.amltmp, 2 files out of an estimated total of 12
Uploading ./automl.ipynb
Uploaded ./automl.ipynb, 3 files out of an estimated total of 12
Uploading ./hyperparameter_tuning (1).ipynb
Uploaded ./hyperparameter_tuning (1).ipynb, 4 files out of an estimated total of 12
Uploading ./hyperparameter_tuning.ipynb
Uploaded ./hyperparameter_tuning.ipynb, 5 files out of an estimated total of 12
Uploading ./model_training.py
Uploaded ./model_training.py, 6 files out of an estimated total of 12
Uploading ./README.md
Uploaded ./README.md, 7 files out of an estimated total of 12
Uploading ./score.py
Uploaded ./score.py, 8 files out of an estimated total of 12
Uploading ./train.py
Uploaded ./train.py, 9 files out of an estimated total of 12
Uploading ./traindata.csv
Uploaded ./traindata.csv, 10 files out of an estimated total of 12


In [19]:
temp = ftrain_data.drop_columns('Column1')

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [20]:
# TODO: Put your automl settings here
automl_settings = {    
    "experiment_timeout_minutes": 25,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted'}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
                             compute_target=comp_trget,
                             task = "classification",
                             training_data=temp,
                             label_column_name="Churn",   
                             enable_early_stopping= True,
                             featurization= 'auto',
                             n_cross_validations=7,
                             debug_log = "automl_errors.log",
                             **automl_settings
)

In [21]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output = True)


Submitting remote run.
No run_configuration provided, running on automl-cluster with default configuration
Running on remote compute: automl-cluster


Experiment,Id,Type,Status,Details Page,Docs Page
capstone-project,AutoML_78a45b32-8739-4c0b-a14d-2ce6e71df070,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

******************************************************************

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [22]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
capstone-project,AutoML_78a45b32-8739-4c0b-a14d-2ce6e71df070,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more abo

{'runId': 'AutoML_78a45b32-8739-4c0b-a14d-2ce6e71df070',
 'target': 'automl-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-09-26T13:46:48.381314Z',
 'endTimeUtc': '2021-09-26T14:13:54.25214Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '7',
  'target': 'automl-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"capstone-project","subscription_id":"61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30","resource_group":"aml-quickstarts-159333","workspace_name":"quick-starts-ws-159333","region":"southcentralus","compute_target":"automl-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"start_auxiliary_runs_before_parent_complete":false,"enable_code_generation":false,"iterations":1000,"primary_metric

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [23]:
best_automl, best_fit_model = remote_run.get_output()
print(best_fit_model)

Package:azureml-automl-runtime, training version:1.34.0.post1, current version:1.33.0
Package:azureml-core, training version:1.34.0, current version:1.33.0
Package:azureml-dataprep, training version:2.22.2, current version:2.20.1
Package:azureml-dataprep-rslex, training version:1.20.2, current version:1.18.0
Package:azureml-dataset-runtime, training version:1.34.0, current version:1.33.0
Package:azureml-defaults, training version:1.34.0, current version:1.33.0
Package:azureml-interpret, training version:1.34.0, current version:1.33.0
Package:azureml-mlflow, training version:1.34.0, current version:1.33.0
Package:azureml-pipeline-core, training version:1.34.0, current version:1.33.0
Package:azureml-responsibleai, training version:1.34.0, current version:1.33.0
Package:azureml-telemetry, training version:1.34.0, current version:1.33.0
Package:azureml-train-automl-client, training version:1.34.0, current version:1.33.0
Package:azureml-train-automl-runtime, training version:1.34.0.post1, c

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
), random_state=0, reg_alpha=0, reg_lambda=0.10416666666666667, subsample=0.7, tree_method='auto'))], verbose=False)), ('13', Pipeline(memory=None, steps=[('minmaxscaler', MinMaxScaler(copy=True, feature_range=(0, 1))), ('extratreesclassifier', ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight='balanced', criterion='gini', max_depth=None, max_features=0.9, max_leaf_nodes=None, max_samples=None, min_impurity_decrease=0.0, min_impurity_split=None, min_samples_leaf=0.01, min_samples_split=0.01, min_weight_fraction_leaf=0.0, n_estimators=25, n_jobs=1, oob_score=False, random_state=None, ve

In [24]:
print(best_automl)

Run(Experiment: capstone-project,
Id: AutoML_78a45b32-8739-4c0b-a14d-2ce6e71df070_55,
Type: azureml.scriptrun,
Status: Completed)


In [25]:
#TODO: Save the best model
joblib.dump(best_fit_model, 'best_fit_automl_model.pkl')

['best_fit_automl_model.pkl']

In [18]:
#joblib.dump(best_fit_model,'./outputs/bestmodel.joblib')
#best_class.register_model(model_name='best_fit_automl_model.pkl',model_path='/outputs/')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [26]:
from azureml.core.model import Model
model = Model.register(workspace = ws, model_name = 'best_fit_automl_model', model_path = 'best_fit_automl_model.pkl')
print(model.name, model.id, model.version, sep='\t')

Registering model best_fit_automl_model
best_fit_automl_model	best_fit_automl_model:1	1


TODO: In the cell below, send a request to the web service you deployed to test it.

In [27]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import LocalWebservice, Webservice, AciWebservice
from azureml.core.conda_dependencies import CondaDependencies
import azureml.train.automl

# Create the environment
env = best_automl.get_environment()
conda_dep = CondaDependencies()

# Define the packages needed by the model and the script
#conda_dep.add_conda_package("numpy")
#conda_dep.add_conda_package("scikit-learn")
#conda_dep.add_conda_package("pandas")
# Define azureml-defaults as a pip dependency
#conda_dep.add_pip_package("azureml-defaults")
#conda_dep.add_pip_package("azureml-train")

# Adds dependencies to PythonSection of myenv
#env.python.conda_dependencies=conda_dep

inference_config = InferenceConfig(entry_script='score.py', environment=env)

deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=4, enable_app_insights=True)
service = Model.deploy(ws, "customerservice", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)

print(service.state)
print(service.scoring_uri)
print(service.swagger_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-09-26 14:19:29+00:00 Creating Container Registry if not exists..
2021-09-26 14:29:29+00:00 Registering the environment.
2021-09-26 14:29:30+00:00 Generating deployment configuration.
2021-09-26 14:29:31+00:00 Submitting deployment to compute..
2021-09-26 14:29:35+00:00 Checking the status of deployment customerservice..
2021-09-26 14:34:37+00:00 Checking the status of inference endpoint customerservice.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy
http://f2d071d7-2a8f-46db-8f35-966004a4f42e.southcentralus.azurecontainer.io/score
http://f2d071d7-2a8f-46db-8f35-966004a4f42e.southcentralus.azurecontainer.io/swagger.json


TODO: In the cell below, print the logs of the web service and delete the service

In [35]:
import requests
import json

# URL for the web service
scoring_uri = 'http://f2d071d7-2a8f-46db-8f35-966004a4f42e.southcentralus.azurecontainer.io/score'

# Set the content type
headers = {'Content-Type': 'application/json'}

# Two sets of data to score, so we get two results back
data = {"data":
        [{
                "gender_Male" :0,
                "SeniorCitizen":1,
                "Dependents" :0,
                "Partner" :0,
                "tenure":2,
                "PhoneService" :1,
                "MultipleLines" :1,
                "InternetService_Fiber optic" :1,
                "InternetService_No":0,
                "OnlineSecurity" :0,
                "OnlineBackup" :1,
                "DeviceProtection" :0,
                "TechSupport" :0,
                "StreamingTV" :0,
                "StreamingMovies" :1,
                "Contract_One year" : 0,
                "Contract_Two year":0,
                "PaperlessBilling" :1,
                "PaymentMethod_Bank transfer (automatic)":0,
                "PaymentMethod_Credit card (automatic)":0,
                "PaymentMethod_Mailed check":0,
                "PaymentMethod_Electronic check" :1,
                "MonthlyCharges" : 55.7,
                "TotalCharges" : 239.8 }
        ]
        }
# Convert to JSON string
input_data = json.dumps(data)

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)

print("Response Code : ", resp.status_code)
print("Predicted Value : ",resp.text)

Response Code :  200
Predicted Value :  [1]


In [36]:
# Web Service Logs
print(service.get_logs())

2021-09-26T14:34:32,852683600+00:00 - iot-server/run 
2021-09-26T14:34:32,851860200+00:00 - gunicorn/run 
Dynamic Python package installation is disabled.
Starting HTTP server
2021-09-26T14:34:32,874007800+00:00 - rsyslog/run 
2021-09-26T14:34:32,968172000+00:00 - nginx/run 
rsyslogd: /azureml-envs/azureml_732632de80c129004c31329a710852e7/lib/libuuid.so.1: no version information available (required by rsyslogd)
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2021-09-26T14:34:33,949403000+00:00 - iot-server/finish 1 0
2021-09-26T14:34:33,951475000+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (78)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 108
SPARK_HOME not set. Skipping PySpark Initialization.
Generating new fontManager, this may take some time...
Initializing logger
2021-09-26 14:34:39,717 | root | INFO | Starting up app insights client
logging socket was

In [26]:
# Delete the service
service.delete()